<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazirmatn" color="#0099cc">
Unsupervised nlp pca svm fake news</font>
</h1>

In [1]:
import numpy as np
import pandas as pd 
import scipy.sparse as ss

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, f1_score

from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split

<center>
<div dir=rtl style="direction: rtl;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazirmatn" size=3>
    
|ستون|توضیحات|
|:------:|:---:|
|author|شخصی که متن خبر را نوشته است|
|published|تاریخ و ساعتی که خبر منتشر شده است|
|site_url|سایت منتشرکننده خبر|
|type|برای هر خبر یک دسته یا یک نوعی نیز ذخیره کرده‌ایم. این ستون می‌تواند یکی از مقادیر `bs` به معنی *پرت* یا ‍‍‍‍`conspiracy` به معنی *توطئه آمیز* یا `bias` به معنی *سوگیری یا متعصبانه* یا `hate` به معنی *نفرت‌افکن یا خشم‌آلود* یا ‍`satrie` به معنی *طنز* یا `junksci` به معنی *به درد نخور یا آشغال* و `fake` به معنی *دروغ* باشد|
|label|برچسب خبر را نشان می‌دهد. این ستون یا `Fake` است یا `Real`|
    
</font>
</div>
</center>



In [2]:
train_data = pd.read_csv('../data/news_train.csv')
train_data_text_vectors = ss.load_npz('../data/news_train_text_vectors.npz')

test_data = pd.read_csv('../data/news_test.csv')
test_data_text_vectors = ss.load_npz('../data/news_test_text_vectors.npz')

In [3]:
test_data_text_vectors

<COOrdinate sparse matrix of dtype 'float64'
	with 52811 stored elements and shape (346, 42141)>

In [4]:
train_data_vectors_df = pd.DataFrame(train_data_text_vectors.toarray())

test_data_vectors_df = pd.DataFrame(test_data_text_vectors.toarray())


In [5]:
test_data_vectors_df.head(1)

,0,1,2,3,4,5,6,7,8,9,...,42131,42132,42133,42134,42135,42136,42137,42138,42139,42140
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
train_data.head(3)

,author,published,site_url,type,label
0,No Author,2016-11-01T03:28:50.389+02:00,clickhole.com,satire,Fake
1,Anonymous,2016-10-27T21:30:00.000+03:00,abeldanger.net,bs,Fake
2,Kali74,2016-10-27T02:54:49.093+03:00,abovetopsecret.com,bs,Fake


In [7]:
train_data.isna().sum()

author       0
published    0
site_url     0
type         0
label        0
dtype: int64

In [8]:
pca = PCA(n_components=1500)
train_vector_reduced = pca.fit_transform(train_data_vectors_df)
test_vector_reduced = pca.transform(test_data_vectors_df)

In [9]:
type(train_vector_reduced)

numpy.ndarray

In [10]:
train_vector_reduced_df = pd.DataFrame(train_vector_reduced)
test_vector_reduced_df = pd.DataFrame(test_vector_reduced)

In [11]:
type(train_vector_reduced_df)

pandas.core.frame.DataFrame

In [12]:
label = train_data['label']
train_data.drop(columns=['label'], inplace=True)

In [13]:
label = label.map({'Fake': 0, 'Real': 1})
label

0       0
1       0
2       0
3       0
4       1
       ..
1495    1
1496    0
1497    0
1498    1
1499    0
Name: label, Length: 1500, dtype: int64

In [14]:
# train_data['published'] = pd.to_datetime(train_data['published'])
# test_data['published'] = pd.to_datetime(test_data['published'])

train_data.drop(columns=['published','author' ], inplace=True)
test_data.drop(columns=['published', 'author'], inplace=True)

In [15]:
enc = OneHotEncoder(handle_unknown='ignore')

train_data_enc = enc.fit_transform(train_data)
test_data_enc = enc.transform(test_data)

In [16]:
train_data_enc_df = pd.DataFrame(train_data_enc.toarray())
test_data_enc_df  = pd.DataFrame(test_data_enc.toarray())

test_data_enc_df.shape

(346, 68)

In [17]:
fixed_train = pd.concat([train_data_enc_df, train_vector_reduced_df], axis=1)
fixed_test  = pd.concat([test_data_enc_df,  test_vector_reduced_df], axis=1)
fixed_train.shape

(1500, 1568)

In [18]:
X_train, X_val, y_train, y_val = train_test_split(fixed_train, label,random_state=42, test_size=0.30)

In [19]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,1490,1491,1492,1493,1494,1495,1496,1497,1498,1499
485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-7.196804e-20,2.151531e-19,2.498902e-19,-1.301228e-19,-4.673832e-20,3.211388e-20,3.430305e-20,-4.857363e-20,-9.289608e-21,-8.380374e-21
527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-7.196804e-20,2.151531e-19,2.498902e-19,-1.301228e-19,-4.673832e-20,3.211388e-20,3.430305e-20,-4.857363e-20,-9.289608e-21,-8.380374e-21
199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-7.196804e-20,2.151531e-19,2.498902e-19,-1.301228e-19,-4.673832e-20,3.211388e-20,3.430305e-20,-4.857363e-20,-9.289608e-21,-8.380374e-21
889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-7.196804e-20,2.151531e-19,2.498902e-19,-1.301228e-19,-4.673832e-20,3.211388e-20,3.430305e-20,-4.857363e-20,-9.289608e-21,-8.380374e-21
844,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-7.196804e-20,2.151531e-19,2.498902e-19,-1.301228e-19,-4.673832e-20,3.211388e-20,3.430305e-20,-4.857363e-20,-9.289608e-21,-8.380374e-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-7.196804e-20,2.151531e-19,2.498902e-19,-1.301228e-19,-4.673832e-20,3.211388e-20,3.430305e-20,-4.857363e-20,-9.289608e-21,-8.380374e-21
1294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-7.196804e-20,2.151531e-19,2.498902e-19,-1.301228e-19,-4.673832e-20,3.211388e-20,3.430305e-20,-4.857363e-20,-9.289608e-21,-8.380374e-21
860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,-7.196804e-20,2.151531e-19,2.498902e-19,-1.301228e-19,-4.673832e-20,3.211388e-20,3.430305e-20,-4.857363e-20,-9.289608e-21,-8.380374e-21
1459,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-7.196804e-20,2.151531e-19,2.498902e-19,-1.301228e-19,-4.673832e-20,3.211388e-20,3.430305e-20,-4.857363e-20,-9.289608e-21,-8.380374e-21


In [20]:
# from sklearn.model_selection import GridSearchCV

In [21]:
model = SVC(
    kernel='rbf'
)

model.fit(X_train, y_train)

SVC()

In [22]:
pred_val = model.predict(X_val)

f1_score(pred_val, y_val)

1.0

In [23]:
test_pred = np.where(model.predict(fixed_test) ==0, "Fake", 'Real')
submission = pd.DataFrame(test_pred, columns=['label'])
submission

,label
0,Real
1,Fake
2,Real
3,Real
4,Real
...,...
341,Fake
342,Real
343,Fake
344,Fake
